In [2]:
import numpy as np
import pandas as pd

In [32]:
import requests
import json

# Data Acqisition

In [19]:
data['results'][0]['overview']

'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.'

In [38]:
temp=[]
for i in range(0,462):
    response=requests.get(f'https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={i}')
    data=response.json()
    
    if 'results' in data:
        for movie in data['results']:
            temp.append((movie['original_title'],movie['overview'],movie['genre_ids']))

In [40]:
df=pd.DataFrame(temp)

In [44]:
df.head()

,movie_title,overview,genre
0,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[18, 80]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,千と千尋の神隠し,"A young girl, Chihiro, becomes trapped in a st...","[16, 10751, 14]"


In [42]:
df.columns=['movie_title','overview','genre']

In [47]:
g_response=requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US')
g_data=g_response.json()

In [60]:
genres=g_data['genres']

In [61]:
def get_genre(genre_ids):
    genre_names=[genre['name'] for genre in genres if genre['id'] in genre_ids]
    return genre_names

In [62]:
df['genre']=df['genre'].apply(get_genre)

In [70]:
df.to_csv('movies_overview.csv')

# Text Preprocessing

In [3]:
df=pd.read_csv('movies_overview.csv')

In [4]:
df=df.dropna()

In [5]:
df['overview'][0]

'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.'

- converting the overviw to lower case in order to avoid wastage of storage

In [6]:
df['overview']=df['overview'].apply(lambda x:x.lower())

- As there are no html tags we can skip the removal of html tags step

- As there are no urls we can skip

- Remove the puncuations
-- we can use regular expression for this

In [7]:
import string
exclude=string.punctuation

In [8]:
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [10]:
df['overview']=df['overview'].apply(remove_punc)

- In the data there are no spelling errors i think

- Remove stop words

In [11]:
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

In [12]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords:
            new_text.append('')
        else:
            new_text.append(word)
    return ' '.join(new_text)

In [13]:
df['overview']=df['overview'].apply(remove_stopwords)

- we dont have any emojis in the data so no emoji handling

- Tokenize the sentence

we can do this using nltk and spacy library. Spacy lib gives accurate tokenization

In [14]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\upend\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
sent1='hello! all this is upendr, from cse upendra@gmail.com'
# sent1=remove_punc(sent1)
word_tokenize(sent1)

['hello',
 '!',
 'all',
 'this',
 'is',
 'upendr',
 ',',
 'from',
 'cse',
 'upendra',
 '@',
 'gmail.com']

In [16]:
import spacy

In [17]:
nlp=spacy.load('en_core_web_sm')

In [18]:
doc1=nlp(sent1)

In [19]:
for token in doc1:
    print(token)

hello
!
all
this
is
upendr
,
from
cse
upendra@gmail.com


In [30]:
def tokenize(text):
    doc=nlp(text)
    new_lst=[]
    for token in doc:
        new_lst.append(token.text)
    return ' '.join(new_lst)

In [31]:
df['overview']=df['overview'].apply(tokenize)

- apply stemming

In [32]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [33]:
def stem_words(text):
    new_lst=[]
    for word in text.split():
        new_lst.append(ps.stem(word))
    return ' '.join(new_lst)

In [34]:
stem_words('pta   conservative high school  indiana bans samesex couples  attending  annual prom  gang  flamboyant broadway stars try  boost  image  showing   support two lesbian students')

'pta conserv high school indiana ban samesex coupl attend annual prom gang flamboy broadway star tri boost imag show support two lesbian student'

In [35]:
df['overview']=df['overview'].apply(stem_words)

In [37]:
df['overview'][0]

'frame 1940 doubl murder wife lover upstand banker andi dufresn begin new life shawshank prison put account skill work amor warden long stretch prison dufresn come admir inmat includ older prison name red integr unquench sens hope'

In [38]:
df.to_csv('preprocessed_movie_data.csv')